In [4]:
from pathlib import Path

import pandas as pd
from codetf.models import load_model_pipeline
from sacrebleu.metrics import BLEU
from datasets import load_dataset

In [5]:
root_dir = Path.cwd()

In [6]:
dataset = load_dataset("json", data_files={
    "test": str(root_dir / "data" / "test.jsonl"),
}, cache_dir=root_dir / "data" / "cache")

Found cached dataset json (/home/paul/projects/edu/master/mdl-ii/src/data/cache/json/default-acdd91729f392843/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 17.94it/s]


In [7]:
def inference(doc):
    doc["ref"] = [" ".join(docstring) for docstring in doc["docstring_tokens"]]
    return doc    

dataset = dataset.map(inference, batched=True)

In [9]:
dataset.set_format(type="pandas", columns=["ref", "code"])
df = dataset["test"][:]
df.head()

,code,ref
0,def sina_xml_to_url_list(xml_data):\n rawur...,str - > list Convert XML to URL List . From Bi...
1,"def dailymotion_download(url, output_dir='.', ...",Downloads Dailymotion videos by URL .
2,"def sina_download(url, output_dir='.', merge=T...",Downloads Sina videos by URL .
3,"def sprint(text, *colors):\n return ""\33[{}...",Format text with color or other effects into A...
4,"def print_log(text, *colors):\n sys.stderr....",Print a log message to standard error .


In [13]:
model = load_model_pipeline(model_name="codet5", task="sum_python", model_type="base")
preds = []

for i, row in df.head(1000).iterrows():
    if i % 100 == 0: print(i)
    preds.append(model.predict([row["code"]]))

0


Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors


100
200
300
400
500
600
700
800


KeyboardInterrupt: 

In [3]:
method = ["""
def hello():
    print("Hello world")
"""]

model = load_model_pipeline(model_name="codet5", task="sum_python", model_type="base")
sys = model.predict(method)

In [4]:
from sacrebleu.metrics import BLEU
bleu = BLEU()
bleu.corpus_score(sys, [["prints hello world"]])

BLEU = 31.95 50.0/33.3/25.0/25.0 (BP = 1.000 ratio = 1.333 hyp_len = 4 ref_len = 3)

In [5]:
bleu.get_signature()

nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1

In [ ]:
method = ["def sina_xml_to_url_list(xml_data):\n    rawurl = []\n    dom = parseString(xml_data)\n    for node in dom.getElementsByTagName('durl'):\n        url = node.getElementsByTagName('url')[0]\n        rawurl.append(url.childNodes[0].data)\n    return rawurl"]